In [1]:
import os
from tensorflow import keras
from datagenerator import *
from metrics import *
from keras.models import load_model
from keras.callbacks import CSVLogger

In [2]:
def dir_to_ids(dir_list):
    x = []
    for i in range(0, len(dir_list)):
        x.append(dir_list[i][dir_list[i].rfind('/')+1:])
    return x

In [3]:
custom_objects = {'specificity': specificity,
                  'sensitivity': sensitivity,
                  'dice_coef': dice_coef,
                  'dice_coef_necrotic': dice_coef_necrotic,
                  'dice_coef_enhancing': dice_coef_enhancing,
                  'dice_coef_edema': dice_coef_edema,
                  'precision': precision}
model = load_model('model/unet_v1.h5', custom_objects=custom_objects, compile=True)

In [4]:
test = dir_to_ids(os.listdir('data/test/'))
test_generator = DataGenerator('data/test/', test)

In [5]:
csv_logger = CSVLogger('log/test.log', separator=',', append=False)
callbacks = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                               factor=0.2,
                                               patience=2,
                                               min_lr=0.000001,
                                               verbose=1), csv_logger]

In [6]:
results = model.evaluate(test_generator, batch_size=10, callbacks=callbacks)

125/125 [==============================] - 1478s 12s/step - loss: 0.0467 - accuracy: 0.9849 - mean_io_u: 0.4093 - dice_coef: 0.3520 - precision: 0.9908 - sensitivity: 0.9804 - specificity: 0.9968 - dice_coef_necrotic: 0.2162 - dice_coef_edema: 0.3347 - dice_coef_enhancing: 0.2603
